In [1]:
import pandas as pd
acs_df = pd.read_csv('../capstone_data/ACS_Data.csv')
print(acs_df[:3])

   YEAR  DATANUM  SERIAL  HHWT  STATEFIP  COUNTYFIPS  CITY  GQ  PERNUM  PERWT  \
0  2012        1       1    74         1           0     0   1       1     73   
1  2012        1       1    74         1           0     0   1       2     45   
2  2012        1       1    74         1           0     0   1       3     90   

    OCC   IND  INCTOT  POVERTY  
0  5700  8090   25000      259  
1  8140  3570   35000      259  
2     0     0       0      259  


In [2]:
acs_df.columns

Index([u'YEAR', u'DATANUM', u'SERIAL', u'HHWT', u'STATEFIP', u'COUNTYFIPS',
       u'CITY', u'GQ', u'PERNUM', u'PERWT', u'OCC', u'IND', u'INCTOT',
       u'POVERTY'],
      dtype='object')

In [3]:
acs_df.head(3)

,YEAR,DATANUM,SERIAL,HHWT,STATEFIP,COUNTYFIPS,CITY,GQ,PERNUM,PERWT,OCC,IND,INCTOT,POVERTY
0,2012,1,1,74,1,0,0,1,1,73,5700,8090,25000,259
1,2012,1,1,74,1,0,0,1,2,45,8140,3570,35000,259
2,2012,1,1,74,1,0,0,1,3,90,0,0,0,259


## gdp by industry

In [1]:
orig_gdp_df = pd.read_csv('../capstone_data/allgmp.csv', dtype = str)

# to add years as column 
years = list(orig_gdp_df.columns[8:])
keep = list(orig_gdp_df.columns[0:8])
gdp_pvtdf = pd.melt(orig_gdp_df, id_vars = keep , value_vars = years, var_name = 'Year', value_name = 'GDP by Metro Area')


#to see how many rows have na values
print(len(gdp_pvtdf), len(gdp_pvtdf.dropna(thresh = 8)))


#to see how many regions have na values
print(len(gdp_pvtdf['Region'].dropna()))
print(gdp_pvtdf['Region'].nunique())

#drop region column
gdp_dfmin = gdp_pvtdf.drop(columns = ['Region', 'IndustryClassification'])

#to see if 'ComponentName' column is necessary
componentname = gdp_dfmin['ComponentName']
print(componentname.nunique())
gdp_dfmin['ComponentName'].unique()
gdp_dfmin['ComponentId'].unique()

#convert number columns to float

gdp_dfmin['GeoFIPS'] = pd.to_numeric(gdp_dfmin['GeoFIPS'], errors = 'coerce')
gdp_dfmin['GDP by Metro Area'] = pd.to_numeric(gdp_dfmin['GDP by Metro Area'], errors = 'coerce')


#to make a df of only nominal gdp and change header
nominalgdp_df = gdp_dfmin[gdp_dfmin['ComponentId']== '200']
nominalgdp_df = nominalgdp_df.rename(columns = {'GDP by Metro Area':'Nominal GDP'})
nominalgdp_df = nominalgdp_df.drop(columns = ['ComponentId', 'ComponentName'])
nominalgdp_df.head()

#to make a df of only real gdp and change header
realgdp_df = gdp_dfmin[gdp_dfmin['ComponentId']== '900']
realgdp_df = realgdp_df.drop(columns = ['ComponentId', 'ComponentName'])
realgdp_df = realgdp_df.rename(columns = {'GDP by Metro Area':'Real GDP'})
realgdp_df.head()

#make a df of industry total real gdp
realgdp_df1 = realgdp_df[realgdp_df['IndustryId']=='1']
realgdp_df1 = realgdp_df1.drop(columns = ['GeoName','IndustryId', 'Description'])
realgdp_df1.head()

#make a dictionary of industry total
my_dict = {}
for index, row in realgdp_df1.iterrows():
    dict2 = {(row['GeoFIPS'], row['Year']): row['Real GDP']}
    my_dict.update(dict2)


#define function that will return percent of total gdp for that year and zip(using dict)
def gdp_percent(row):
    return row['Real GDP']/my_dict[(row['GeoFIPS'], row['Year'])]
    
#use function to create column 'Percent GDP'
realgdp_df['Percent GDP']= realgdp_df.apply(lambda row: gdp_percent(row), axis =1)

#set index as year, geofips, and industry description
ind_gdp = realgdp_df.set_index(['Year','GeoFIPS', 'IndustryId'])


#drop unnessary columns and unstack by industry id
ind_gdp = ind_gdp.drop(columns = ['GeoName', 'Description'])

ind_gdp = ind_gdp.unstack(-1)

#join multilevel columns by '_'
ind_gdp.columns = ['_'.join(col).strip() for col in ind_gdp.columns.values]

#drop US value
ind_gdp = ind_gdp.drop(index = 998.0, level = 1)

ind_gdp = ind_gdp.reset_index()

#convert GeoFIPS to string and mame sure all have a len()==5

ind_gdp['GeoFIPS'] = ind_gdp['GeoFIPS'].astype(str)
for code in ind_gdp['GeoFIPS']:
    if len(code)<5:
        print(code)



NameError: name 'pd' is not defined